In [10]:
import geopandas as gpd
merged_gdf_no_clusters = gpd.read_file('../data/xenium/outs/clusters/stardist_nuc_merged.geojson')
merged_gdf_no_clusters

,id,area,geometry
0,ID_6225,989.890053,"POLYGON ((7147.75562 6836.00000, 7147.47046 68..."
1,ID_6232,372.125690,"POLYGON ((6280.57520 6964.00000, 6280.31128 69..."
2,ID_6235,269.271466,"POLYGON ((7205.94556 6740.00000, 7205.58691 67..."
3,ID_6249,288.273555,"POLYGON ((6448.83301 7252.00000, 6448.27563 72..."
4,ID_6253,527.311285,"POLYGON ((6924.29858 7188.00000, 6923.43896 71..."
...,...,...,...
145658,ID_215215,81.387987,"POLYGON ((24364.99078 21268.00000, 24365.42200..."
145659,ID_215216,152.561310,"POLYGON ((25931.39380 21776.00000, 25931.60718..."
145660,ID_215218,155.985648,"POLYGON ((26654.85815 21128.00000, 26654.66943..."
145661,ID_215219,96.433117,"POLYGON ((25631.47217 21608.00000, 25631.91846..."


In [59]:
import sys
sys.path.append('../src/utils')
import os

# Importing importlib for module reloading
import importlib
import image_cropp_utils

# Reloading the module
importlib.reload(image_cropp_utils)
from image_cropp_utils import load_dataframe

In [60]:
nuc_data = load_dataframe('../data/xenium/outs/nucleus_boundaries.parquet')

In [61]:
nuc_data

,cell_id,vertex_x,vertex_y
0,1,844.900024,324.700012
1,1,843.625000,325.762512
2,1,843.412476,327.250000
3,1,844.900024,328.737488
4,1,844.687500,330.012512
...,...,...,...
2180757,167780,7494.237305,5114.024902
2180758,167780,7492.750000,5112.537598
2180759,167780,7491.475098,5111.687500
2180760,167780,7487.649902,5111.049805


In [65]:
import numpy as np
transform_matrix = np.array([
    [1.7125010585876286, 0.008345081015717404, -10397.19948741904],
    [0.008345081015717404, -1.7125010585876286, 37211.538978661185],
    [0, 0, 1]
])

# Get the inverse matrix
transform_matrix_inv = np.linalg.inv(transform_matrix)

# Extract coordinates and apply scaling
coords = nuc_data[['vertex_x', 'vertex_y']].values
micron_to_pixel = 0.2125  # Microns per pixel
coords_scaled = coords / micron_to_pixel

# Function to transform coordinates
def transform_coordinates(coords, transform_matrix):
    coords = np.hstack((coords, np.ones((len(coords), 1))))
    transformed_coords = np.dot(transform_matrix, coords.T).T
    # Return only the x and y coordinates
    return transformed_coords[:, :2]

# Apply the transformation to the scaled coordinates
transformed_coords = transform_coordinates(coords_scaled, transform_matrix_inv)

# Update the DataFrame with the transformed coordinates
nuc_data[['x_location_transformed', 'y_location_transformed']] = transformed_coords

In [68]:
from shapely.geometry import Polygon, Point
# Create a GeoDataFrame from cells_data
geometry = [Point(xy) for xy in zip(nuc_data['x_location_transformed'], nuc_data['y_location_transformed'])]
cells_gdf = gpd.GeoDataFrame(nuc_data, geometry=geometry)
cells_gdf.crs = None

In [69]:
joined = gpd.sjoin(merged_gdf_no_clusters, cells_gdf, how='left', predicate='intersects')

/var/folders/fd/jw2fvp1178s0spb_1sy9f51h0000gn/T/ipykernel_55663/3109362285.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  joined = gpd.sjoin(merged_gdf_no_clusters, cells_gdf, how='left', predicate='intersects')


In [71]:
filtered_joined = joined[~joined['index_right'].isna()]

In [72]:
filtered_joined

,id,area,geometry,index_right,cell_id,vertex_x,vertex_y,x_location_transformed,y_location_transformed
0,ID_6225,989.890053,"POLYGON ((7147.75562 6836.00000, 7147.47046 68...",549572.0,42285.0,397.799988,5438.512695,7131.260874,6819.295013
0,ID_6225,989.890053,"POLYGON ((7147.75562 6836.00000, 7147.47046 68...",549573.0,42285.0,401.200012,5437.875000,7140.595247,6821.092323
0,ID_6225,989.890053,"POLYGON ((7147.75562 6836.00000, 7147.47046 68...",549571.0,42285.0,393.762512,5436.174805,7120.135019,6825.665290
1,ID_6232,372.125690,"POLYGON ((6280.57520 6964.00000, 6280.31128 69...",558437.0,42967.0,85.212502,5387.087402,6271.615155,6956.420849
1,ID_6232,372.125690,"POLYGON ((6280.57520 6964.00000, 6280.31128 69...",558438.0,42967.0,86.699997,5387.087402,6275.702629,6956.440768
...,...,...,...,...,...,...,...,...,...
145639,ID_215173,182.948379,"POLYGON ((24196.42700 20768.00000, 24197.07355...",1963526.0,151068.0,6627.662598,395.462494,24182.730506,20760.481150
145639,ID_215173,182.948379,"POLYGON ((24196.42700 20768.00000, 24197.07355...",1963535.0,151068.0,6631.062500,395.462494,24192.073344,20760.526678
145639,ID_215173,182.948379,"POLYGON ((24196.42700 20768.00000, 24197.07355...",1963537.0,151068.0,6628.937500,394.612488,24186.222688,20762.833932
145639,ID_215173,182.948379,"POLYGON ((24196.42700 20768.00000, 24197.07355...",1963525.0,151068.0,6628.937500,394.612488,24186.222688,20762.833932


In [76]:
joined.to_file("../data/xenium/outs/clusters/mapping_nucleus_to_cell2.geojson", driver='GeoJSON')